<a href="https://colab.research.google.com/github/seobando/TradingBot/blob/main/4_kpis_interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/1 Formal Training/2 Msc. Ciencia de los datos/Semestre IV/Trader

/content/drive/MyDrive/1 Formal Training/2 Msc. Ciencia de los datos/Semestre IV/Trader


In [ ]:
import random
import torch
import pandas as pd
import numpy as np
from collections import deque

from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("data/data_daily_^XAU.csv").drop("Unnamed: 0",axis=1)

In [ ]:
df.describe()

,Open,High,Low,Close,Volume,MA_short,MA_long,EMA_short,EMA_long,EMA_short_MACD,EMA_long_MACD,MACD,MACD_signal,MACD_hist,BIAS_short,BIAS_long,VR,OBV
count,1259.000000,1259.000000,1259.000000,1259.000000,1259.0,1210.000000,1060.000000,1259.000000,1259.000000,1259.000000,1259.000000,1259.000000,1251.000000,1251.000000,1210.000000,1060.000000,0.0,1259.0
mean,102.154400,103.390929,100.897458,102.133820,0.0,101.841871,100.769899,101.347157,98.589300,101.960728,101.739282,0.221446,0.224431,-0.001805,0.952030,4.653852,NaN,0.0
std,28.494284,28.885170,28.042782,28.479337,0.0,28.094925,26.906467,27.433830,24.093710,28.193422,27.894825,2.225135,2.072235,0.741419,7.577944,13.816675,NaN,0.0
min,61.380001,62.020000,60.590000,61.840000,0.0,65.916600,69.892300,66.266865,72.128377,64.147773,65.387167,-8.002896,-6.248162,-3.382668,-30.834610,-25.263087,NaN,0.0
25%,80.564999,81.555001,79.965000,80.579998,0.0,81.744600,79.960975,82.227845,82.978032,81.429659,81.794770,-1.086611,-0.990670,-0.351329,-3.898739,-5.114570,NaN,0.0
50%,89.959999,91.129997,88.820000,89.620003,0.0,87.830600,85.071600,89.190383,86.750754,89.533788,89.066378,0.128991,0.090952,0.012155,0.442492,1.818529,NaN,0.0
75%,131.455001,133.100006,129.709999,131.389999,0.0,131.645000,131.100250,131.306278,126.279779,131.644373,131.227904,1.307799,1.248991,0.396869,4.845352,14.354593,NaN,0.0
max,166.000000,167.089996,164.539993,166.009995,0.0,154.497200,145.335650,155.012718,143.498382,162.722209,159.200034,8.338662,7.599237,3.417444,28.433653,49.357583,NaN,0.0


In [ ]:
def kpis_conversion(df,change_col,compare_col):

  if type(compare_col) == str:

    conditions  = [ df[change_col] == df[compare_col], df[change_col] > df[compare_col] , df[change_col] < df[compare_col] ]
    choices     = [ 0, 1, -1 ]
    change_col_name = change_col + "_interpreted"
    df[change_col_name] = np.select(conditions, choices, default=np.nan)

  else:

    conditions  = [ df[change_col] == 0, df[change_col] > 0 , df[change_col] < 0 ]
    choices     = [ 0, 1, -1 ]
    change_col_name = change_col + "_interpreted"
    df[change_col_name] = np.select(conditions, choices, default=np.nan)

  return  df

In [ ]:
def kpis_interpretation(df):

  # Open
  df = kpis_conversion(df,"Open","Close")
  # High
  df = kpis_conversion(df,"High","Close")
  # Low
  df = kpis_conversion(df,"Low","Close")
  # MA_short
  df = kpis_conversion(df,"MA_short","Close")  
  # MA_long
  df = kpis_conversion(df,"MA_long","Close")  
  # EMA_short
  df = kpis_conversion(df,"EMA_short","Close")
  # EMA_long
  df = kpis_conversion(df,"EMA_long","Close")
  # MACD
  df = kpis_conversion(df,"MACD","MACD_signal")
  # BIAS_short
  df = kpis_conversion(df,"BIAS_short",0)
  # BIAS_long
  df = kpis_conversion(df,"BIAS_long",0)
  # VR
  df = kpis_conversion(df,"VR","Volume")
  # OBV
  df = kpis_conversion(df,"OBV","Volume")

  return df

In [ ]:
df = kpis_interpretation(df)
df.to_csv("data/data_daily_interpreted_^XAU.csv")
df.head()

,Date,Open,High,Low,Close,Volume,MA_short,MA_long,EMA_short,EMA_long,EMA_short_MACD,EMA_long_MACD,MACD,MACD_signal,MACD_hist,BIAS_short,BIAS_long,VR,OBV,Open_interpreted,High_interpreted,Low_interpreted,MA_short_interpreted,MA_long_interpreted,EMA_short_interpreted,EMA_long_interpreted,MACD_interpreted,BIAS_short_interpreted,BIAS_long_interpreted,VR_interpreted,OBV_interpreted
0,2017-01-17,88.580002,89.800003,88.570000,89.099998,0,NaN,NaN,89.099998,89.099998,89.099998,89.099998,0.000000,NaN,NaN,NaN,NaN,NaN,0,-1.0,1.0,-1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0
1,2017-01-18,89.040001,90.029999,86.940002,87.959999,0,NaN,NaN,89.055292,89.088655,88.924614,89.015554,-0.090940,NaN,NaN,NaN,NaN,NaN,0,1.0,1.0,-1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,0.0
2,2017-01-19,86.919998,88.400002,86.400002,87.559998,0,NaN,NaN,88.996653,89.073444,88.714673,88.907735,-0.193062,NaN,NaN,NaN,NaN,NaN,0,-1.0,1.0,-1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,0.0
3,2017-01-20,87.669998,89.809998,87.300003,88.680000,0,NaN,NaN,88.984235,89.069529,88.709338,88.890865,-0.181527,NaN,NaN,NaN,NaN,NaN,0,-1.0,1.0,-1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,0.0
4,2017-01-23,89.599998,91.309998,89.269997,91.260002,0,NaN,NaN,89.073481,89.091325,89.101748,89.066357,0.035391,NaN,NaN,NaN,NaN,NaN,0,-1.0,1.0,-1.0,NaN,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,0.0


In [ ]:
## QA
required_fields = ['Open_interpreted', 'High_interpreted','Low_interpreted', 'MA_short_interpreted', 'MA_long_interpreted', 'EMA_short_interpreted','EMA_long_interpreted', 'MACD_interpreted', 'BIAS_short_interpreted', 'BIAS_long_interpreted', 'VR_interpreted', 'OBV_interpreted']
# Set selected indicators
df = df[required_fields]
# Stack table
df_stacked = df.melt(var_name='indicators', value_name='marks')
df_stacked = df_stacked.groupby(['indicators', 'marks']).size().reset_index().rename(columns={0:"values"})
# Get indicators per mark
df_pivot = (
    df_stacked
            .pivot_table(
                index = "indicators"
                , columns = "marks" 
                , values = "values"
                , aggfunc = "sum"
                ,margins = True
                ,fill_value = np.NaN)
            .reset_index()
            .rename(columns={-1.0:"minor",0.0:"equal",1.0:"major"})
            )
# Count null values
df_pivot["levels"] = df_pivot.isnull().sum(axis=1)
display(df_pivot)

marks,indicators,minor,equal,major,All,levels
0,BIAS_long_interpreted,480.0,NaN,580.0,1060,1
1,BIAS_short_interpreted,568.0,NaN,642.0,1210,1
2,EMA_long_interpreted,668.0,1.0,590.0,1259,0
3,EMA_short_interpreted,670.0,1.0,588.0,1259,0
4,High_interpreted,NaN,20.0,1239.0,1259,1
5,Low_interpreted,1244.0,15.0,NaN,1259,1
6,MACD_interpreted,609.0,NaN,642.0,1251,1
7,MA_long_interpreted,580.0,NaN,480.0,1060,1
8,MA_short_interpreted,642.0,NaN,568.0,1210,1
9,OBV_interpreted,NaN,1259.0,NaN,1259,2


In [ ]:
# Get indicators with issues
with_issues = df_pivot[df_pivot["levels"]==2]
display(with_issues)

marks,indicators,minor,equal,major,All,levels
9,OBV_interpreted,NaN,1259.0,NaN,1259,2
